In the previous notebook we created a workflow made up of two agents, both driven by an LLM, one acting as writer and the other checking
 the produced content and providing feedback or approving.  
 
But what if instead of using an LLM for evaluating the poem we want a human to provide feedback and/or approve?
We can use a special agent that acts as a sort of Human agent.

In [9]:
import sys
sys.path.append("..")
from model_clients.azure import get_model
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from rich import print

In the following cell, we'll define two agents, one, as before its in charge of creating the content,
 while the other is a special `UserProxyAgent` 
 that acts as any other agent but will handle the input through a dedicate input function.  
 
In this specific example it will map to Python's `input`function to allow manual entry.

In [16]:
# Create the writer agent.
writer_agent = AssistantAgent(
    "writer",
    model_client=get_model(),
    system_message="You are a helpful AI assistant.",
)

# Create the human/user agent.
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.


Now we create the team and run it.

In [20]:
termination = TextMentionTermination("APPROVE")

# Create the team.
team = RoundRobinGroupChat([writer_agent, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
result= await Console(stream)
print("[cyan][bold]--- Conversation complete! ---[/bold][/cyan]")
print(result)

# Note how a team (as an agent)can save its state to a JSON object.
# This JSON object can be used to re-create the team's state later using the `load_state` method.
json=(await team.save_state())
print(json)

---------- user ----------
Write a 4-line poem about the ocean.
---------- writer ----------
The ocean's breath, a gentle sigh,  
Its waves embrace the starlit sky.  
A world of wonder, vast and free,  
Home to dreams and memory.
---------- user_proxy ----------
make it in shakeseare style
---------- writer ----------
Upon the vast and endless sea,  
Where waves do kiss the starry plea,  
A realm of wonder doth unfold,  
Guarding dreams and tales untold.
---------- user_proxy ----------
APPROVE


--- Conversation complete! ---

TaskResult(
    messages=[
        TextMessage(
            source='user',
            models_usage=None,
            metadata={},
            content='Write a 4-line poem about the ocean.',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=209, completion_tokens=37),
            metadata={},
            content="The ocean's breath, a gentle sigh,  \nIts waves embrace the starlit sky.  \nA world of wonder,
vast and free,  \nHome to dreams and memory.",
            type='TextMessage'
        ),
        UserInputRequestedEvent(
            source='user_proxy',
            models_usage=None,
            metadata={},
            request_id='907aa1e5-1b07-428e-a9a8-c5e0ca111ebe',
            content='',
            type='UserInputRequestedEvent'
        ),
        TextMessage(
            source='user_proxy',
            models_usage=None,
            metadata={},
            content='make it in shakeseare style',
            type='TextMessage'
        ),
        TextMessage(
            source='writer',
            models_usage=RequestUsage(prompt_tokens=264, completion_tokens=36),
            metadata={},
            content='Upon the vast and endless sea,  \nWhere waves do kiss the starry plea,  \nA realm of wonder 
doth unfold,  \nGuarding dreams and tales untold.',
            type='TextMessage'
        ),
        UserInputRequestedEvent(
            source='user_proxy',
            models_usage=None,
            metadata={},
            request_id='ec9932c8-f400-4f11-9109-c345a011932b',
            content='',
            type='UserInputRequestedEvent'
        ),
        TextMessage(source='user_proxy', models_usage=None, metadata={}, content='APPROVE', type='TextMessage')
    ],
    stop_reason="Text 'APPROVE' mentioned"
)

{
    'type': 'TeamState',
    'version': '1.0.0',
    'agent_states': {
        'writer': {
            'type': 'ChatAgentContainerState',
            'version': '1.0.0',
            'agent_state': {
                'type': 'AssistantAgentState',
                'version': '1.0.0',
                'llm_context': {
                    'messages': [
                        {
                            'content': 'Write a 4-line poem about the ocean.',
                            'source': 'user',
                            'type': 'UserMessage'
                        },
                        {
                            'content': 'Beneath the waves, where secrets lie,  \nEndless blue that meets the sky.  
\nWhispers of the deep, a tranquil song,  \nWhere dreams and tides both belong.',
                            'thought': None,
                            'source': 'writer',
                            'type': 'AssistantMessage'
                        },
                        {
                            'content': 'Write a 4-line poem about the ocean.',
                            'source': 'user',
                            'type': 'UserMessage'
                        },
                        {
                            'content': 'Endless horizon, vast and deep,  \nWhere mysteries and wonders sleep.  
\nRhythmic waves in ceaseless motion,  \nTell the timeless tale of the ocean.',
                            'thought': None,
                            'source': 'writer',
                            'type': 'AssistantMessage'
                        },
                        {
                            'content': 'please translate it in italian',
                            'source': 'user_proxy',
                            'type': 'UserMessage'
                        },
                        {
                            'content': "Orizzonte infinito, vasto e profondo,  \nDove i misteri e le meraviglie 
riposano.  \nOnde ritmiche in moto incessante,  \nNarrano la storia senza tempo dell'oceano.",
                            'thought': None,
                            'source': 'writer',
                            'type': 'AssistantMessage'
                        },
                        {
                            'content': 'Write a 4-line poem about the ocean.',
                            'source': 'user',
                            'type': 'UserMessage'
                        },
                        {
                            'content': "The ocean's breath, a gentle sigh,  \nIts waves embrace the starlit sky.  
\nA world of wonder, vast and free,  \nHome to dreams and memory.",
                            'thought': None,
                            'source': 'writer',
                            'type': 'AssistantMessage'
                        },
                        {'content': 'make it in shakeseare style', 'source': 'user_proxy', 'type': 'UserMessage'},
                        {
                            'content': 'Upon the vast and endless sea,  \nWhere waves do kiss the starry plea,  \nA
realm of wonder doth unfold,  \nGuarding dreams and tales untold.',
                            'thought': None,
                            'source': 'writer',
                            'type': 'AssistantMessage'
                        }
                    ]
                }
            },
            'message_buffer': [
                {
                    'source': 'user_proxy',
                    'models_usage': None,
                    'metadata': {},
                    'content': 'APPROVE',
                    'type': 'TextMessage'
                }
            ]
        },
        'user_proxy': {
            'type': 'ChatAgentContainerState',
            'version': '1.0.0',
            'agent_state': {'type': 'BaseState', 'version': '1.0.0'},
            'message_buffer': []
        },
        'RoundRobinGroupChatManager': {
            'type': 'RoundRobinManagerStat